# KPI

In [2]:
# import libraries 
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

## Data Loading

In [18]:
PATH = 'Data'
SPRINT = 3
FILENAME = f'Tourist_Accommodation07042025.csv'

In [4]:
# define function to read file
def read_file(filename):
    
    """Input: filename.
    Function reads .csv data. 
    Output: Dataframe or error.
    """
    
    # reading file
    try:
        df = pd.read_csv(os.path.join('..', PATH, filename), 
                         parse_dates=['insert_date', 'first_review_date', 'last_review_date'])
        display(df.head())
        return df
    except Exception as error:
        print("An exception occurred:", error)
        return error 

In [5]:
df = read_file(FILENAME)

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,standardized_city,standardized_neighbourhood,autonomous_community,has_reviews
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,NaN,Private room,2,2.0,1.0,...,100.0,0,75.0,spain,malaga,2018-07-31,Málaga,Centro,Andalucía,True
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,C�rmenes,Latina,Private room,1,1.0,1.0,...,90.0,0,52.0,spain,madrid,2020-01-10,Madrid,Los Cármenes,Comunidad de Madrid,True
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1.0,2.0,...,100.0,1,142.0,spain,sevilla,2019-07-29,Sevilla,San Vicente,Andalucía,True
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,l'Antiga Esquerra de l'Eixample,Eixample,Private room,2,2.0,1.0,...,90.0,1,306.0,spain,barcelona,2020-01-10,Barcelona,L'Antiga Esquerra de l'Eixample,Cataluña,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,NaN,Private room,5,1.0,2.0,...,100.0,0,39.0,spain,girona,2019-02-19,Girona,Quart,Cataluña,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9650 entries, 0 to 9649
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   apartment_id                 9650 non-null   int64         
 1   name                         9647 non-null   object        
 2   description                  9516 non-null   object        
 3   host_id                      9650 non-null   int64         
 4   neighbourhood_name           9650 non-null   object        
 5   neighbourhood_district       5860 non-null   object        
 6   room_type                    9650 non-null   object        
 7   accommodates                 9650 non-null   int64         
 8   bathrooms                    9578 non-null   float64       
 9   bedrooms                     9580 non-null   float64       
 10  beds                         9605 non-null   float64       
 11  amenities_list               9634 non-null 

## KPI occupancy rates
### 1. Occupancy Rate (Monthly) 

Occupancy Rate = (Number of Occupied Days / Number of Days) × 100

In [7]:
# occupancy rate for different periods
for period in ['30', '60', '90', '365']:
    print(f'\nOccupancy Rate {period} days:')
    print(
        ((int(period) - df[f"availability_{period}"]) / int(period)).mean()*100)


Occupancy Rate 30 days:
56.50466321243523

Occupancy Rate 60 days:
52.00639032815199

Occupancy Rate 90 days:
48.73436960276339

Occupancy Rate 365 days:
48.59751579246221


In [8]:
# 1. Occupancy Rate for 30 days
df['occupancy_rate'] = (30 - df.availability_30).div(30).mul(100)

occupancy_rate = df['occupancy_rate'].mean()

# print results
print(f"Occupancy Rate for 30 days: {occupancy_rate:.2f}")

Occupancy Rate for 30 days: 56.50


### 2. City with the Highest Occupancy (Monthly) 

City with the Highest Occupancy = max (Number of Occupied Days / Number of Available Days)


In [9]:
df.groupby('standardized_city').occupancy_rate.mean().sort_values()

standardized_city
Sevilla      49.817814
Girona       50.273850
Menorca      52.293578
Palma        53.198568
Valencia     55.263158
Málaga       57.281746
Barcelona    60.020841
Madrid       60.905967
Name: occupancy_rate, dtype: float64

In [10]:
# 2. City with the Highest Occupancy
best_city = df.groupby('standardized_city').occupancy_rate.mean().idxmax()

# print results
print(f"City with the Highest Occupancy: {best_city}")

City with the Highest Occupancy: Madrid


## KPI customers satisfaction

### 3. General Satisfaction Index (average review_scores_value)

In [11]:
general_satisfaction_index = df['review_scores_rating'].mean()
print(f"General Satisfaction Index: {general_satisfaction_index:.2f}")

General Satisfaction Index: 918.97


### 4. Compute the average score for each category

In [12]:
avg_scores = {
    'accuracy': df['review_scores_accuracy'].mean(),
    'cleanliness': df['review_scores_cleanliness'].mean(),
    'checkin': df['review_scores_checkin'].mean(),
    'communication': df['review_scores_communication'].mean(),
    'location': df['review_scores_location'].mean(),
    'value': df['review_scores_value'].mean() ## Added to cover all columns of review_scores
}

# Find the category with the highest average satisfaction
highest_satisfaction_item = max(avg_scores, key=avg_scores.get)

# Print results
print(f"Highest Satisfaction Item: {highest_satisfaction_item} ({avg_scores[highest_satisfaction_item]:.2f})")

Highest Satisfaction Item: checkin (96.22)


In [13]:
avg_scores

{'accuracy': 94.49258836944128,
 'cleanliness': 93.10595272002278,
 'checkin': 96.22022536014833,
 'communication': 96.2008547008547,
 'location': 95.43651925820257,
 'value': 91.3509272467903}

## Save KPIs to .xlsx

In [21]:
# load existing KPIs data from the Excel file
kpi_data = pd.read_excel(os.path.join('..', 'Results', "KPIs.xlsx"))

In [22]:
kpi_data

,Sprint,1_OccupancyRate,2_HighestOccupancy,3_Satisfaction Index,4_HighestSatisfactionItem,4_Score
0,1,59.03,Madrid,919.99,communication,96.43
1,2,58.65,Madrid,919.31,communication,96.34


In [23]:
# new row with updated KPIs
new_row = {
    "Sprint": SPRINT,
    "1_OccupancyRate": round(occupancy_rate, 2),
    "2_HighestOccupancy": best_city,
    "3_Satisfaction Index": round(general_satisfaction_index, 2),
    "4_HighestSatisfactionItem": highest_satisfaction_item,
    "4_Score": round(avg_scores[highest_satisfaction_item], 2)
}

In [24]:
# concatenate the new row to the existing DataFrame
kpi_data = pd.concat([kpi_data, pd.DataFrame([new_row])], ignore_index=True)
kpi_data

,Sprint,1_OccupancyRate,2_HighestOccupancy,3_Satisfaction Index,4_HighestSatisfactionItem,4_Score
0,1,59.03,Madrid,919.99,communication,96.43
1,2,58.65,Madrid,919.31,communication,96.34
2,3,56.50,Madrid,918.97,checkin,96.22


In [25]:
# Save the updated DataFrame back to the Excel file
kpi_data.to_excel(os.path.join('..', 'Results', "KPIs.xlsx"), index=False)